In [4]:
!pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !pip install torchinfo

In [1]:
import os

import torch
from torchinfo import summary

from monai.networks.nets import UNETR
from monai.config import print_config

print_config()

2023-08-09 21:00:22.929297: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-09 21:00:22.952047: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 21:00:23.390026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


MONAI version: 1.3.dev2332
Numpy version: 1.24.3
Pytorch version: 2.0.1+cu117
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: a260ff4abbb7161c325666baf9ddc8bd4c079031
MONAI __file__: /media/wsshiro/CCA032A5A03295C4/VRAMexp/vramexp/.venv/lib/python3.10/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.1.0
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
scipy version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 10.0.0
Tensorboard version: 2.13.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.15.2+cu117
tqdm version: 4.65.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.0.3
einops version: 0.6.1
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VER

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
    in_channels=1,
    out_channels=14,
    img_size=(96, 96, 96),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to(device)

summary(model, (1,1,96,96,96),
        col_names=["input_size",
                "output_size",
                "num_params",
                "params_percent",
                "kernel_size",
                "mult_adds",
                "trainable"])

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Param %                   Kernel Shape              Mult-Adds                 Trainable
UNETR                                              [1, 1, 96, 96, 96]        [1, 14, 96, 96, 96]       --                             --                   --                        --                        True
├─ViT: 1-1                                         [1, 1, 96, 96, 96]        [1, 216, 768]             --                             --                   --                        --                        True
│    └─PatchEmbeddingBlock: 2-1                    [1, 1, 96, 96, 96]        [1, 216, 768]             165,888                     0.18%                   --                        --                        True
│    │    └─Sequential: 3-1                        [1, 1, 96, 96, 96]        [1, 216, 768]             3,146,496                   3.39%           

In [3]:
print("パッチの要素数", (96/16)**3)

m = 92784046*4
x = 96*96*96*4
y = 1338.16/2*1024**2 # Forwad/backward pass size = total output * 2
batch = 1
b=1
g=m
d=1
o=2
print("推定容量", (m + (x+y)*batch*b + g*d + m*o)/1024**2, "MiB")

パッチの要素数 216.0
推定容量 2088.227186279297 MiB


In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = UNETR(
    in_channels=1,
    out_channels=2,
    img_size=(64, 64, 64),
    feature_size=16,
    hidden_size=768,
    mlp_dim=3072,
    num_heads=12,
    pos_embed="perceptron",
    norm_name="instance",
    res_block=True,
    dropout_rate=0.0,
).to("cpu")

summary(model, (1,1,64, 64, 64),
        col_names=["input_size",
                "output_size",
                "num_params",
                "params_percent",
                "kernel_size",
                "mult_adds",
                "trainable"])

Layer (type:depth-idx)                             Input Shape               Output Shape              Param #                   Param %                   Kernel Shape              Mult-Adds                 Trainable
UNETR                                              [1, 1, 64, 64, 64]        [1, 2, 64, 64, 64]        --                             --                   --                        --                        True
├─ViT: 1-1                                         [1, 1, 64, 64, 64]        [1, 64, 768]              --                             --                   --                        --                        True
│    └─PatchEmbeddingBlock: 2-1                    [1, 1, 64, 64, 64]        [1, 64, 768]              49,152                      0.05%                   --                        --                        True
│    │    └─Sequential: 3-1                        [1, 1, 64, 64, 64]        [1, 64, 768]              3,146,496                   3.40%           

In [5]:
m = 92667106*4
x = 64*64*64*4
y = 371.33/2*1024**2 # Forwad/backward pass size = total output * 2
batch = 1
b=1
g=m
d=1
o=2 # Adam
print("推定容量", (m + (x+y)*batch*b + g*d + m*o)/1024**2, "MiB")

推定容量 1600.652823486328 MiB
